Thymio mini project

In [1]:
!pip install --upgrade tdmclient

In [1]:
# import libraries
import time
from tdmclient import ClientAsync
import numpy as np
import math

# import project files
import constants as cst
import extended_Kalman_filter as eKf
import vision
import geometry as geo


-1.5707963267948966


In [2]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node df054775-a7a5-454e-9ad3-7896a54bbcc8

In [3]:
# Set the motor speeds
async def stop_robot():
    """
    Stop the robot. Set the motor speed of each wheel to zero.
    """
    await node.set_variables({
            "motor.left.target": [0],
            "motor.right.target": [0],
        })

async def move_robot(left_speed, right_speed):
    """
    Turn the robot wheels to the given speeds.
    Parameter: - left_speed    : left wheel speed in [thymio speed]
               - right_speed   : right wheel speed in [thymio speed]
    """
    await node.set_variables({
            "motor.left.target": [left_speed],
            "motor.right.target": [right_speed],
        })

In [4]:
# Get the proximity sensor values
async def get_prox_sensors():
    """
    Get the sensor measurements of a the proximity sensors.
    Returns: - prox_sensors    : numpy array of fron proximity sensors measurements
             - detected        : boolean, true if any sensors measured a higher value than threshold
    """
    
    await node.wait_for_variables({str('prox.horizontal')})
    prox_sensors = np.asarray(list(node['prox.horizontal']))
    prox_sensors = prox_sensors[0:5]

    prox_sensors_bool = (prox_sensors >= np.ones(5) * 50) #cst.THRESHOLD_PROX_SENSOR
    detected = False
    if(any(prox_sensors_bool)):
        detected = True

    return prox_sensors, detected

# Get the motor speed measurement values
async def get_motor_speed_meas():
    """
    Get the motor speed measurements
    Returns: - speed_left_meas    : left wheel speed in [thymio speed]
             - speed_right_meas   : right wheel speed in [thymio speed]
    """
    
    await node.wait_for_variables({str('motor.left.speed')})
    speed_left_meas = node['motor.left.speed']
    await node.wait_for_variables({str('motor.right.speed')})
    speed_right_meas = node['motor.right.speed']

    return speed_left_meas, speed_right_meas

In [5]:
# test thymio
# move the robot forward and get prox values.
await move_robot(50, 50)
dt = 0
old_time = time.time()
while(dt < 2):
    y,d,i = await get_prox_sensors()
    print(y)
    print(d)
    print(i)
    await client.sleep(0.1)
    dt = time.time() - old_time
await stop_robot()

[0 0 0 0 0]
False
-1
[0 0 0 0 0]
False
-1
[0 0 0 0 0]
False
-1
[0 0 0 0 0]
False
-1
[0 0 0 0 0]
False
-1
[0 0 0 0 0]
False
-1
[0 0 0 0 0]
False
-1
[0 0 0 0 0]
False
-1
[0 0 0 0 0]
False
-1
[0 0 0 0 0]
False
-1
[0 0 0 0 0]
False
-1
[0 0 0 0 0]
False
-1
[0 0 0 0 0]
False
-1
[0 0 0 0 0]
False
-1
[0 0 0 0 0]
False
-1
[0 0 0 0 0]
False
-1
[0 0 0 0 0]
False
-1


In [6]:

# state variables
global_state = 'rotation'

### initialisation
# vision (détection robot, obstacles, goal)
#obstacles, goal_centroid, robot_centroid, robot_direction = vision.vision_obstacles_positions()
# global nav

intermediate_goals = [(0, 30), (30, 30)]
robot = [0.0,0.0]
robot_orientation = 0
robot_speed = [0.0,0.0]

Kfilter = eKf.Kalman(robot[0], robot[0], robot_orientation)


old_time = 0.0

iter = 0
while(True):
    print(iter)
    if intermediate_goals == []:
        await stop_robot()
        print("stop robot")
        break

    # vision et gobale toutes les n iterations (i modulo n == 0)
    
    # get robot sensor values
    #prox_sensors, detected = await get_prox_sensors()
    # test if local (with proximity)
    #if(detected):
    #    global_state = 'local_avoidance'
    
    if iter == 0:                   # first time
        old_time = time.time()
    dt = time.time() - old_time     # get time difference for Kalman
    # print("dt: ", dt)
    # Kalman (with wheel speed and camera position)
    old_time = time.time()

    #robot[0] = robot[0] + dt * ((robot_speed[0] + robot_speed[1])/(2.0*18.1818)) * math.cos(robot_orientation)
    #robot[1] = robot[1] + dt * ((robot_speed[0] + robot_speed[1])/(2.0*18.1818)) * math.sin(robot_orientation)
    #robot_orientation = robot_orientation + dt * (robot_speed[0] - robot_speed[1])/(4.0*cst.WHEELS_DIST)
    #print(robot[0]," ", robot[1]," ", robot_orientation)
    
    Kfilter.dt_update(dt)
    speed_left_meas, speed_right_meas = await get_motor_speed_meas()
    Kfilter.measurement_wheels(speed_left_meas / 18.1818, speed_right_meas / 18.1818)
    Kfilter.Kalman_filter()
    print("Kalman: ", Kfilter.Mu)

    if global_state == 'rotation':
        angle = Kfilter.Mu[2] - np.arctan2(intermediate_goals[0][1] - Kfilter.Mu[1], intermediate_goals[0][0] - Kfilter.Mu[0])
        #angle = robot_orientation - np.arctan2(intermediate_goals[0][1] - robot[1], intermediate_goals[0][0] - robot[0])
        print("rotation")
        #print("rotation :", robot_orientation, np.arctan2(intermediate_goals[0][1] - robot[1], intermediate_goals[0][0] - robot[0]))
        
        if abs(angle) < 0.1:
            global_state = 'translation'
            continue

        # call rotation fct
        if(angle > 0):
            robot_speed = [-20, 20]
        else:
            robot_speed = [20, -20]
        await move_robot(robot_speed[0], robot_speed[1])
        

    elif global_state == 'translation':
        print("translation ", intermediate_goals)
        #print("translation: ", robot[0], " ", robot[1], "  ", math.dist(intermediate_goals[0], robot))
        # if goal reached
            # exit
        #if abs(intermediate_goals[0][0] - Kfilter.Mu[0]) < 1 and abs(intermediate_goals[0][1] - Kfilter.Mu[1]) < 1:
        if abs((intermediate_goals[0][0] - Kfilter.Mu[0])**2 + (intermediate_goals[0][1] - Kfilter.Mu[1])**2) < 1:
        #if abs((intermediate_goals[0][0] - robot[0])**2 - (intermediate_goals[0][1] - robot[1])**2) < 2:
            intermediate_goals.pop(0)
            global_state = 'rotation'
            continue
        
        # call translation fct
        robot_speed = [50, 50]
        await move_robot(robot_speed[0], robot_speed[1])

    elif global_state == 'local_avoidance':
        # call local_avoidance fct (= add obstacle on global map and recompute global nav)
        
        global_state = 'rotation'
    time.sleep(0.05)

    iter = iter + 1

0
Kalman:  [[0.        ]
 [0.        ]
 [0.        ]
 [0.88000194]
 [0.66000145]]
rotation
1
Kalman:  [[0.41070411]
 [0.        ]
 [0.00997202]
 [0.74800107]
 [0.26400026]]
rotation
2
Kalman:  [[ 4.40579851e-01]
 [ 2.13063582e-04]
 [ 2.23340658e-02]
 [ 9.30770410e-01]
 [-6.76924406e-01]]
rotation
3
Kalman:  [[ 4.49983966e-01]
 [ 4.51212544e-04]
 [ 3.80024592e-02]
 [ 1.00132467e+00]
 [-9.38236657e-01]]
rotation
4
Kalman:  [[ 4.54572049e-01]
 [ 6.12567066e-04]
 [ 5.50794631e-02]
 [ 1.09629335e+00]
 [-1.03820351e+00]]
rotation
5
Kalman:  [[ 4.35757391e-01]
 [-1.84410280e-04]
 [ 7.42260588e-02]
 [ 9.96610417e-01]
 [-1.41631077e+00]]
rotation
6
Kalman:  [[ 0.4142488 ]
 [-0.00193965]
 [ 0.0945229 ]
 [ 1.02651743]
 [-1.18682903]]
rotation
7
Kalman:  [[ 0.41004239]
 [-0.00244914]
 [ 0.11174084]
 [ 1.00394902]
 [-0.99719873]]
rotation
8
Kalman:  [[ 0.41376554]
 [-0.00206682]
 [ 0.12936911]
 [ 1.02932058]
 [-0.95875836]]
rotation
9
Kalman:  [[ 0.41218443]
 [-0.00219253]
 [ 0.1465717 ]
 [ 0.97102

In [9]:
await stop_robot()